In [1]:
import numpy as np
import h5py
import torch
#from torch.utils.data import Dataset, DataLoader
# import torch.utils.data as data
import torch.utils.data as data_utils
from torch.utils.data.dataset import random_split
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

/Users/wilsonchang/Desktop/Project/test/.env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# helper functions

# make minibatch for training
def minibatch(X, y, batch_size=50):
    X = X.data.numpy()
    y = y.data.numpy()
    N = X.shape[0]
    batch_mask = np.random.choice(N, batch_size)
    X_batch = X[batch_mask]
    y_batch = y[batch_mask]
    return Variable(torch.Tensor(X_batch)), Variable(torch.Tensor(y_batch))

Helper Clases

In [3]:
class EEGDataset(data_utils.Dataset):
    """EEG dataset."""
    def __init__(self, X, y):
        self.X = X;
        self.y = y;
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
class Flatten(nn.Module):
    def forward(self, x):
        N, E, T = x.size() 
        return x.view(N, -1)

Load Data

In [4]:
A01T = h5py.File('project_datasets/A01T_slice.mat', 'r')
print (A01T)
X = np.copy(A01T['image'])
y = np.copy(A01T['type'])
X = X[:, 0:23, :] # only goes to 23 since you dont use last 3 channels
# X = np.moveaxis(X, -1, 0)
y = y[0,0:X.shape[0]:1]
#y = np.asarray(y, dtype=np.int32)
#subtract 769 so classes start at 0
y = y - 769
print (np.shape(X))

<HDF5 file "A01T_slice.mat" (mode r)>
(288, 23, 1000)


In [5]:
data = EEGDataset(X,y)
dset = {}
dset['train'], dset['val'], dset['test'] = random_split(data, [len(data)-100, 50, 50])
#num_workers for GPU stuff
#dataloaders = {x: data_utils.DataLoader(dset[x], batch_size=10, shuffle=True, num_workers=1) for x in ['train', 'val', 'test']}
dataloaders = {x: data_utils.DataLoader(dset[x], batch_size=50, shuffle=True) for x in ['train', 'val', 'test']}

channels = 23
classes = 4
#for i_batch, sample_batched in enumerate(dataloaders['train']):
#    X, y = sample_batched
#    print (X)

In [6]:
# toy data
#N, E, T= 200, 10, 100
#C = 4
#X_train = Variable(torch.randn(T, N, E))

#y = np.random.randint(0, 4, size=100)
#y = Variable(torch.Tensor(y), requires_grad=False)
#y_train = Variable(torch.LongTensor(N).random_(0, C-1))
#actual = y_train.data.numpy()
#X.type(dtype)
#y.type(dtype)

In [7]:
dtype = torch.DoubleTensor
model = nn.Sequential(
    nn.LSTM(channels, classes, 2)
)
model.type(dtype)
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(model.parameters(), lr=1e-2)

In [8]:
#training

num_epochs = 10
batch_size = 10

#iterations_per_epoch = N//batch_size
best_val_acc = 0
train_acc_at_best_val = 0

#for epoch in range(1):
for epoch in range(num_epochs):
    #for t in range(iterations_per_epoch):
    for i, sample_batched in enumerate(dataloaders['train']):
        
        X_train, y_train = sample_batched
        # Flip axis first
        X_train = torch.Tensor(np.moveaxis(X_train.numpy(), -1, 0))
        X_train = Variable(X_train, requires_grad=True).type(dtype)
        y_train = Variable(y_train).long()
        
        # Calculate the loss
        scores, _ = model(X_train)
        _, y_pred = torch.max(scores, 0)
        y_pred = Variable(y_pred.data, requires_grad=True)
        y_pred = y_pred.type(dtype)
        loss = loss_fn(y_pred, y_train)
        print('(%d batch) loss: %f' % (i, loss))
        # Do backpropagation
        model.zero_grad()
        loss.backward()
        # Update the parameters
        optimizer.step()
        ## Training Accuracy After 1 epoch
    y_pred_tot = []
    y_train_tot = []
    for i_batch, sample_batched in enumerate(dataloaders['train']):
        X_train, y_train = sample_batched
        # Flip axis first
        X_train = torch.Tensor(np.moveaxis(X_train.numpy(), -1, 0))
        X_train = Variable(X_train, requires_grad=True).type(dtype)
        y_train = Variable(y_train).long()
        # Calculate the loss
        scores, _ = model(X_train)
        _, y_pred = torch.max(scores, 0)
        _, y_pred = torch.max(y_pred, 1)
        y_pred_tot.append(y_pred.data.numpy())
        y_train_tot.append(y_train.data.numpy())    
    train_acc = np.mean(np.concatenate(y_pred_tot) == np.concatenate(y_train_tot))
        ## Validation Accuracy After 1 epoch    
    y_pred_tot = []
    y_val_tot = []
    for i_batch, sample_batched in enumerate(dataloaders['val']):
        X_val, y_val = sample_batched
        # Flip axis first
        X_val = torch.Tensor(np.moveaxis(X_val.numpy(), -1, 0))
        X_val = Variable(X_val, requires_grad=True).type(dtype)
        y_val = Variable(y_val).long()
        # Calculate the loss
        scores, _ = model(X_val)
        _, y_pred = torch.max(scores, 0)
        _, y_pred = torch.max(y_pred, 1)
        y_pred_tot.append(y_pred.data.numpy())
        y_val_tot.append(y_val.data.numpy())
    val_acc = np.mean(np.concatenate(y_pred_tot) == np.concatenate(y_val_tot))
    print('(Epoch %d / %d) train acc: %f; val_acc: %f' % (epoch+1, num_epochs, train_acc, val_acc))
        

(0 batch) loss: 259.769315
(1 batch) loss: 349.201723
(2 batch) loss: 363.309315
(3 batch) loss: 291.227014
(Epoch 1 / 10) train acc: 0.250000; val_acc: 0.240000
(0 batch) loss: 330.561589
(1 batch) loss: 369.215714
(2 batch) loss: 330.647854
(3 batch) loss: 214.720692
(Epoch 2 / 10) train acc: 0.250000; val_acc: 0.240000
(0 batch) loss: 342.140128
(1 batch) loss: 257.667854
(2 batch) loss: 286.475452
(3 batch) loss: 404.380854
(Epoch 3 / 10) train acc: 0.250000; val_acc: 0.240000
(0 batch) loss: 360.634125
(1 batch) loss: 251.643178
(2 batch) loss: 316.040128
(3 batch) loss: 349.072963
(Epoch 4 / 10) train acc: 0.250000; val_acc: 0.240000
(0 batch) loss: 310.877175
(1 batch) loss: 272.840128
(2 batch) loss: 334.507726
(3 batch) loss: 362.352440
(Epoch 5 / 10) train acc: 0.250000; val_acc: 0.240000
(0 batch) loss: 341.807988
(1 batch) loss: 300.403306
(2 batch) loss: 326.353863
(3 batch) loss: 296.115429
(Epoch 6 / 10) train acc: 0.250000; val_acc: 0.240000
(0 batch) loss: 341.227854
(

In [9]:
y_pred = model(X_train)
loss = loss_fn(last_output, y_train)
print (loss)
model

NameError: name 'last_output' is not defined